# Crypto Playground

The notebook where I learn crypto, time series, technical analysis, etc and develop my initial models.

<a id="toc"></a>

# Table of Contents

- [Import Libraries](#import-libraries)
- [Configuration](#configuration)
- [Load Data](#load-data)
- [Missing Values](#missing-values)
- [Summary Statistics](#summary-statistics)



# Sources

1. [Crypto Prediction: Technical Analysis Features](https://www.kaggle.com/yamqwe/crypto-prediction-technical-analysis-features)
2. [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition) - The Starter Notebook
3. [volatility plots](https://www.kaggle.com/pashadude/volatility-plots)
4. [G-Research - Plotly/Cufflinks Interactive Plots](https://www.kaggle.com/dschettler8845/g-research-plotly-cufflinks-interactive-plots)
5. [Feature Engineering: Rolling Aggregations](https://www.kaggle.com/yamqwe/feature-engineering-rolling-aggregations/notebook)
6. [PurgedGroupTimeSeries CV with Extra Data | NN](https://www.kaggle.com/yamqwe/purgedgrouptimeseries-cv-with-extra-data-nn)
7. [Crypto Prediction: XGB Regressor](https://www.kaggle.com/yamqwe/crypto-prediction-xgb-regressor)
8. [💲 G-Research Plots + EDA 💲](https://www.kaggle.com/odins0n/g-research-plots-eda)
9. [Crypto Forecasting - Intro EDA](https://www.kaggle.com/lucasmorin/crypto-forecasting-intro-eda)
10. [Simple Exploration Notebook - Crypto Forecasting](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-crypto-forecasting)

# Time Series Tutorials

- https://www.kaggle.com/iamleonie/time-series-interpreting-acf-and-pacf
- https://www.kaggle.com/iamleonie/intro-to-time-series-forecasting

<a id="import-libraries"></a>

# **<span style="color:#d5a500;">Import Libraries</span>**

 - [TOC](#toc)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go # Candlesticks


from datetime import datetime # timestamp support


<a id="configuration"></a>

# **<span style="color:#d5a500;">Configuration</span>**

In [ ]:
data_dir = Path('../input/g-research-crypto-forecasting')
!ls $data_dir

# Date and Time Utilities

## totimestamp()

- https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition?scriptVersionId=78721864&cellId=35

In [ ]:
# https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition?scriptVersionId=78721864&cellId=35
import time

# auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

## Example of How to Use Method

In [ ]:
totimestamp("1/1/2018")

<a id="load-data"></a>

# **<span style="color:#d5a500;">Load Data</span>**

## Datatable

Loads the data faster but I'm not familar with it so I'm still doing to use Pandas too.

In [ ]:
%%time

import datatable as dt

dt = dt.fread(data_dir / "train.csv")

In [ ]:
dt

## Load with Pandas

In [ ]:
%%time
dtype={'Asset_ID': 'int8', 'Count': 'int32', 'row_id': 'int32', 'Count': 'int32',
       'Open': 'float32', 'High': 'float32', 'Low': 'float32', 'Close': 'float32',
       'Volume': 'float32', 'VWAP': 'float32'}

# train_df = pd.read_csv('../input/g-research-crypto-forecasting/train.csv', low_memory=False, parse_dates=['date'], dtype=dtype)
train_df = pd.read_csv( data_dir / "train.csv", low_memory=False, parse_dates=True)

In [ ]:
train_df.info()

# Asset Details

In [ ]:
asset_details = pd.read_csv(data_dir / "asset_details.csv")
asset_details

# Working with Bitcoin

In [ ]:
bitcoin = train_df[train_df["Asset_ID"] == 1]
bitcoin.head(3)

In [ ]:
bitcoin.head()

In [ ]:
train_df.loc[train_df.High < 14052.3 , "timestamp"] # ['timestamp'] #.fromtimestamp(timestamp).strftime("%d/%m/%y")

In [ ]:
train_df.loc[2, 'timestamp']

# Unix Date Format Conversion

- https://stackoverflow.com/questions/3682748/converting-unix-timestamp-string-to-readable-date

In [ ]:
datetime.fromtimestamp(train_df.loc[2, 'timestamp']).strftime("%d/%m/%Y %H:%M:%S") # 

In [ ]:
bitcoin.iloc[2] # ['timestamp'] #.fromtimestamp(timestamp).strftime("%d/%m/%y")

In [ ]:
bitcoin.loc[2,'timestamp'] #.fromtimestamp(timestamp).strftime("%d/%m/%y")

# Get 2017 Bitcoin Data

In [ ]:
totimestamp("1/1/2018")

In [ ]:
bitcoin_2021 = bitcoin.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]
bitcoin_2020 = bitcoin.loc[totimestamp('01/01/2020'):totimestamp('01/01/2021')]


In [ ]:
bitcoin_2020

In [ ]:
# datetime.fromtimestamp(btc2017.index[1].timestamp).strftime("%d/%m/%y")

In [ ]:
bitcoin = bitcoin.set_index("timestamp")
bitcoin_last7 = bitcoin.iloc[-7:]
bitcoin_last30 = bitcoin.iloc[-30:]
bitcoin_last200 = bitcoin.iloc[-200:]

In [ ]:
bitcoin_last7

In [ ]:
print(f"{bitcoin_last7.iloc[0]['Open']}, {bitcoin_last7.iloc[0]['Close']}")

In [ ]:
# bitcoin_last7.index[0].fromtimestamp(timestamp).strftime("%d/%m/%y")

# Last 7 Dates

In [ ]:
# fromtimestamp(timestamp).strftime("%d/%m/%y")

In [ ]:
datetime.fromtimestamp(train_df.loc[2, 'timestamp']).strftime("%d/%m/%Y %H:%M:%S")

In [ ]:
bitcoin.index

In [ ]:
bitcoin_last7.reset_index(inplace=True)

In [ ]:
bitcoin_last7

In [ ]:
bitcoin_last7['Date'] = bitcoin_last7.loc[:,'timestamp'].apply(datetime.fromtimestamp)
bitcoin_last7

In [ ]:
# datetime.fromtimestamp(bitcoin_last7.timestamp).strftime("%d/%m/%Y %H:%M:%S")

In [ ]:
# bitcoin_last7['date'] = datetime.fromtimestamp(bitcoin_last7.timestamp).strftime("%d/%m/%Y %H:%M:%S")
# bitcoin_last7.head()

# Candlestick Charts

## References

- https://www.investopedia.com/trading/candlestick-charting-what-is-it/
- https://www.investopedia.com/articles/active-trading/092315/5-most-powerful-candlestick-patterns.asp
- https://en.wikipedia.org/wiki/Candlestick_pattern

In [ ]:
fig = go.Figure(data=[go.Candlestick(x = bitcoin_last7.Date, 
                                     open=bitcoin_last7['Open'],
                                     high=bitcoin_last7['High'], 
                                     low=bitcoin_last7['Low'], 
                                     close=bitcoin_last7['Close'])])
fig.show()

# Last 30 Dates

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=bitcoin_last30.index, 
                                     open=bitcoin_last30['Open'],
                                     high=bitcoin_last30['High'], 
                                     low=bitcoin_last30['Low'], 
                                     close=bitcoin_last30['Close'])])
fig.show()

In [ ]:
bitcoin.index[0].astype('datetime64[s]')

In [ ]:
begin_bitcoin = bitcoin.index[0].astype('datetime64[s]')
end_bitcoin = bitcoin.index[-1].astype('datetime64[s]')

print('Bitcoin data goes from ', begin_bitcoin, 'to ', end_bitcoin)

In [ ]:
bitcoin_last30.Close.plot();

In [ ]:
# parse_dates=[['Date','Time']]
plt.figure(figsize=(12,8))
plt.plot(bitcoin_last30.Low)
plt.plot(bitcoin_last30.High)
plt.xlim([0,0.5])
# xlim(0,0.5)
plt.show()

# Open/High/Low/Close Chart

In [ ]:
 ohlc = go.Figure(data = [go.Ohlc(x=bitcoin_last7.index, 
                                               open = bitcoin_last7[('Open')], 
                                               high = bitcoin_last7[('High')], 
                                               low = bitcoin_last7[('Low')], 
                                               close = bitcoin_last7[('Close')])])
ohlc.update_xaxes(title_text = 'Time',
                             rangeslider_visible = False)

In [ ]:
# f = plt.figure(figsize=(7,8))

# ax = f.add_subplot(211)
# plt.plot(btc_mini_2021['Close'], label='btc')
# plt.legend()
# plt.xlabel('Time')
# plt.ylabel('Bitcoin Close')

# ax2 = f.add_subplot(212)
# ax2.plot(eth_mini_2021['Close'], color='red', label='eth')
# plt.legend()
# plt.xlabel('Time')
# plt.ylabel('Ethereum Close')

# plt.tight_layout()
# plt.show()

# Technical Analysis

Traditional Stock Trading Analysis applied to Crypto

- MACD
- Moving Average
  - SMA
  - 50 Day MA
  - 200 Day MA
- Volatility (Variance)
  - Bollinger Bands- Bollinger Bands
- Momentum Indicators
  - Momentum Rate Of Change (ROCMomentum Rate Of Change (ROC
  - RSI- RSI
  - Stochastic Oscillator MomentumStochastic Oscillator Momentum